### Analisi Dati Facebook

1. pip install facebook-sdk: https://github.com/mobolic/facebook-sdk, https://facebook-sdk.readthedocs.io/en/latest/
    
2. Tool di Esplorazioni API Graph: https://developers.facebook.com/tools/explorer/145634995501895/#_=_
    
3. ID Pagina Facebook: http://findmyfbid.com/

In [1]:
import facebook
import os
import requests
import pandas as pd

# Leggo Configuration File
config = {}
config_path = os.path.join(os.environ['HOME'], 'opendata')
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)
token=config['TOKEN_FB']

# Utilizzo token generoato manualmente con la mia utenza sul Tool di Esplorazioni API Graph
graph = facebook.GraphAPI(access_token=token, version='2.7')

In [2]:
# TEST

# Esempio di chiamata su Tool Esplorazione: me?fields=id,name -> Result è un Dict (casomai Dataframe Pandas)
info = graph.get_object(id='me')
info

{'id': '10153605453803372', 'name': 'Andrea Cesarini'}

In [9]:
# PAGINA FACEBOOK

# Recupero ID della Pagina di Salvini (https://www.facebook.com/salviniofficial/?fref=ts) da findmyfbid 
# ID Pagina Salvini 252306033154
# ID Pagina Renzi: 113335124914
# ID Pagina M5S: 174457180812
info_s = graph.get_object(id='174457180812', fields='name,fan_count,posts')

In [10]:
# NUMERO FAN

# Recupero il numero di Fan (sono quelli che mettono mi piace, non è detto che siano quelli che lo seguono in
# quanto uno mette mi piace anche solo per seguirlo)
num_fan = info_s['fan_count']
num_fan

956775

# 1. Distribuzione likes ai posts in base alla data dei post (aggregati per anno / mese)
## Crescita social delle pagine politiche -> In base alla data, legarle ad avvenimenti

In [11]:
# Recupero TUTTI i posts sulla pagina XXXX
posts = graph.get_connections(id='174457180812', connection_name='posts')
all_posts = {}
while(True):
    try:
        for post in posts['data']:
            all_posts[post['id']]=post['created_time']
        posts=requests.get(posts['paging']['next']).json()
    except KeyError:
        break
len(all_posts)

21931

In [12]:
#all_posts

In [13]:
df = pd.DataFrame(list(all_posts.items()), columns=['ID', 'Post_Date'])

In [14]:
df.head(5)

,ID,Post_Date
0,174457180812_10151715791785813,2013-07-17T17:20:00+0000
1,174457180812_10152502294540813,2014-07-22T17:39:19+0000
2,174457180812_10152000804565813,2013-11-26T16:00:42+0000
3,174457180812_10152615088325813,2014-09-08T12:14:51+0000
4,174457180812_10152216213355813,2014-03-08T17:00:24+0000


In [15]:
# Salvo il DF per non ripetere la Chiamata
dir_df = os.path.join(os.path.abspath(''),'stg')
df_filename = r'df_posts_m5s.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df.to_pickle(df_fullpath)